In [1]:
import utils.dir_utils as dir_utils
import utils.dict_utils as dict_utils
import utils.ptr_utils as ptr_utils
import utils.constants as constants 
import utils.profit as profit 
import helpers.official as official
import utils.csv_utils as csv_utils
import statistics
from random import sample

_, input_df = dir_utils.get_data(combined=True)

In [2]:
def setup():
    # (t[constants.TDATE], official.get_name(t), t[constants.TICKER])
    purchases = set()
    
    for _,t in input_df.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            core =  (t[constants.TDATE], official.get_name(t), t[constants.TICKER])
            
            if t[constants.TYPE] == 'Purchase' and ptr_utils.get_year(t[constants.TDATE]) > 2019: 
                purchases.add(core)

    return purchases

p = setup()
print(len(p))

6004


In [5]:
# 3 hours.
def get_purch_price(sample_p):
    # 30 days == 4.28  ~~~~~~~ 45 days == 6.42  ~~~~~~~~~~~ 7. 
    b_0 = []
    b_7 = []
    b_14 = []
    i = 0 
        
    for (tdate_purch, name_purch, ticker_purch) in sample_p:
        print(i)
        i += 1 
        
        tdate_purch_edited = profit.add_to_date(tdate_purch, 0)
        profit_0 = profit.get_stock_price(ticker_purch, tdate_purch_edited)
        b_0.append( [name_purch, ticker_purch, tdate_purch, tdate_purch_edited, profit_0] )

        tdate_purch_edited = profit.add_to_date(tdate_purch, 7)
        profit_7 = profit.get_stock_price(ticker_purch, tdate_purch_edited)
        b_7.append( [name_purch, ticker_purch, tdate_purch, tdate_purch_edited, profit_7] )

        tdate_purch_edited = profit.add_to_date(tdate_purch, 14)
        profit_14 = profit.get_stock_price(ticker_purch, tdate_purch_edited)
        b_14.append( [name_purch, ticker_purch, tdate_purch, tdate_purch_edited, profit_14] )

        if i == 5: break 
    return b_0, b_7, b_14

b_0, b_7, b_14 = get_purch_price(p)
# 345m

0
1
2
3
4


In [6]:
def match(p_prime):
    # 6 months == 26 == short?
    # 1 year == ~long
    # 2 years == long
    sell = [26, 52]
    res = []
    i = 0 
    for (name_purch, ticker_purch, tdate_purch, tdate_purch_edited, purch_price )  in p_prime:

            for increment_sale in sell: 
                tdate_sale = profit.add_to_date(tdate_purch_edited, increment_sale)
                sale, purch, x = profit.share_diff(ticker_purch, tdate_sale, purch_price)
            
                res.append([name_purch, ticker_purch, tdate_purch, tdate_sale, tdate_purch_edited,x])
                
            print(i)
            i += 1
            
    return res

# 0.7s

In [7]:
def rowify(something):
    copi = something.copy()
    res = {}

    while copi:
        name_purch, ticker_purch, tdate_purch, tdate_sale, tdate_purch_edited, x  = copi.pop()
        
        
        if (name_purch, ticker_purch, tdate_purch) not in res:
            res = dict_utils.increment_dictionary(res, (name_purch, ticker_purch, tdate_purch), (tdate_purch_edited, tdate_sale, x), not_math=True)
        else: 
            old = res[(name_purch, ticker_purch, tdate_purch)]
            res = dict_utils.increment_dictionary(res, (name_purch, ticker_purch, tdate_purch), (old, tdate_sale, x), not_math=True)

    rows = []
    for (name, ticker_purch, tdate_purch), v in res.items():
        row = [name, ticker_purch, tdate_purch]

        ((a, c, z),  e, f) = v  

        row.append(a)
        row.append(c)
        row.append(z)

        row.append(e)
        row.append(f)

        rows.append(row)

    return rows 
# 0.4 s 

In [8]:
def get_average(res):
    
    l = {"52" : [], 
         "26" : []}
    
    for [name, ticker_purch, tdate_purch, b, s_52, p_52, s_26, p_26] in res:
        
        # if p_104:
        #     l["104"].append(round(p_104,4))
        if p_52:
            l["52"].append(round(p_52, 4))
        if p_26: 
            l["26"].append(round(p_26, 4))

    for k,v in l.items():
        l[k] = round(statistics.mean(v), 4)
    return l 

def get_people_average(res):
    # person avg(p1), avg(p2)
    people = {}
    for [name, ticker_purch, tdate_purch, b, s_52, p_52, s_26, p_26] in res:
    
        if p_52: 
            people = dict_utils.increment_list_in_inner_dictionary(people, name, "52", p_52)
        if p_26: 
            people = dict_utils.increment_list_in_inner_dictionary(people, name, "26", p_26)

    finalized = []
    
    for person, d in people.items():
        r = [person]
        for v in d:
            r.append(round(statistics.mean(d[v]), 4))
            
        finalized.append(r)
            
    
    return finalized 

def get_ticker_average(res):
    people = {}
    for [name, ticker_purch, tdate_purch, b, s_52, p_52, s_26, p_26] in res:
        
        if p_52: 
            people = dict_utils.increment_list_in_inner_dictionary(people, ticker_purch, "52", p_52)
        if p_26: 
            people = dict_utils.increment_list_in_inner_dictionary(people, ticker_purch, "26", p_26)

    finalized = []
    
    for person, d in people.items():
        r = [person]
        for v in d:
            r.append(round(statistics.mean(d[v]), 4))
            
        finalized.append(r)
            
    
    return finalized 
# 0.5 s 

In [11]:
print(len(b_0))
matched_b_0 = match(b_0)
rowify_b_0 = rowify(matched_b_0)
dir = dir_utils.makesubdir(constants.path_csv, constants.PROFIT+"/2019")
csv_utils.make_csv_base(dir, "2019_b0", ['name', 'ticker_purch', 'tdate_purch', 'bought_?', 's_52', 'p_52' , 's_26', 'p_26'], rowify_b_0)

get_average_res = get_average(rowify_b_0)
csv_utils.make_csv(dir, "2019_b0"+"_get_average",  get_average_res, ["", "Average"])

get_people_average_res = get_people_average(rowify_b_0)
csv_utils.make_csv_base(dir, "2019_b0"+"_get_people_average", [constants.OFFICIAL, "52", "26",], get_people_average_res)

get_ticker_average_res = get_ticker_average(rowify_b_0)
csv_utils.make_csv_base(dir, "2019_b0"+"_get_ticker_average", [constants.TICKER, "52", "26",], get_ticker_average_res)

# 5 Hours

5
0
1
2


In [10]:
print(len(b_7))
matched_b_7 = match(b_7)
rowify_b_7 = rowify(matched_b_7)
dir = dir_utils.makesubdir(constants.path_csv, constants.PROFIT+"/2019")
csv_utils.make_csv_base(dir, "2019_b7", ['name', 'ticker_purch', 'tdate_purch', 'bought_?', 's_104',	'p_104', 's_52', 'p_52' , 's_26', 'p_26'], rowify_b_7)

get_average_res = get_average(rowify_b_7)
csv_utils.make_csv(dir, "2019_b7"+"_get_average",  get_average_res, ["", "Average"])

get_people_average_res = get_people_average(rowify_b_7)
csv_utils.make_csv_base(dir, "2019_b7"+"_get_people_average", [constants.OFFICIAL,  "104", "52", "26",], get_people_average_res)

get_ticker_average_res = get_ticker_average(rowify_b_7)
csv_utils.make_csv_base(dir, "2019_b7"+"_get_ticker_average", [constants.TICKER, "104", "52", "26",], get_ticker_average_res)

5
0
1
2
3


KeyboardInterrupt: 

In [ ]:
print(len(b_14))
matched_b_14 = match(b_14)
rowify_b_14 = rowify(matched_b_14)
dir = dir_utils.makesubdir(constants.path_csv, constants.PROFIT+"/2019")
csv_utils.make_csv_base(dir, "2019_b_14", ['name', 'ticker_purch', 'tdate_purch', 'bought_?', 's_104',	'p_104', 's_52', 'p_52' , 's_26', 'p_26'], rowify_b_14)

get_average_res = get_average(rowify_b_14)
csv_utils.make_csv(dir, "2019_b_14"+"_get_average",  get_average_res, ["", "Average"])

get_people_average_res = get_people_average(rowify_b_14)
csv_utils.make_csv_base(dir, "2019_b_14"+"_get_people_average", [constants.OFFICIAL,  "104", "52", "26",], get_people_average_res)

get_ticker_average_res = get_ticker_average(rowify_b_14)
csv_utils.make_csv_base(dir, "2019_b_14"+"_get_ticker_average", [constants.TICKER, "104", "52", "26",], get_ticker_average_res)